# This solver works in testing for puzzles below 'hard' from sudoku.com, by detecting 'solo candidates' and 'hidden singles'
### 
#### To avoid confusion, "super squares" refer to the nine sets of 3x3 boxes. The 81 boxes which make a sudoku grid are referred to as "elements".
#### 
#### A potential answer refers to the possible answers an element may have after eliminating the answers already found in the given super square, row, or column.
#### 
#### A solo candidate is simply when an element has only one potential answer https://sudoku.com/sudoku-rules/obvious-singles/ 
####  
#### A hidden single is when an element has multiple potential answers, however one number amongst those potential answers appears only once in a super square, row, or column https://sudoku.com/sudoku-rules/hidden-singles/ 

In [3]:
import copy
# the following four functions chart the relationships between columns, rows, super squares, and their elements
def identify_row_or_super_square_element(column, row):
    if column in [0,3,6]:
        if row in [0,3,6]: super_square_element = 0            
        if row in [1,4,7]: super_square_element = 3            
        if row in [2,5,8]: super_square_element = 6            
            
    if column in [1,4,7]:
        if row in [0,3,6]: super_square_element = 1            
        if row in [1,4,7]: super_square_element = 4            
        if row in [2,5,8]: super_square_element = 7            
            
    if column in [2,5,8]:
        if row in [0,3,6]: super_square_element = 2            
        if row in [1,4,7]: super_square_element = 5            
        if row in [2,5,8]: super_square_element = 8
            
    return super_square_element
                              
def column_routine__identify_super_square(column, row):
    if column in [0,1,2]:
        if row in [0,1,2]: super_square = 0
        if row in [3,4,5]: super_square = 3
        if row in [6,7,8]: super_square = 6            
            
    if column in [3,4,5]:
        if row in [0,1,2]: super_square = 1 
        if row in [3,4,5]: super_square = 4
        if row in [6,7,8]: super_square = 7            
            
    if column in [6,7,8]:
        if row in [0,1,2]: super_square = 2            
        if row in [3,4,5]: super_square = 5            
        if row in [6,7,8]: super_square = 8
            
    return super_square

# the following is used for two purposes which coincidentally have the same relationship:
# 1. determines the column given a super square and super square element 
# e.g., super square 2's element 5 is column 8, or super square 6's element 7 is row 1
# 2. determines the super square element given a row and column
# e.g., row 7, column 4 is super square element 4 (of super square 7)
def identify_column_or_super_square_element(super_square_or_row, super_square_element_or_column):
    if super_square_or_row in [0,3,6]:
        if super_square_element_or_column in [0,3,6]: column_or_super_square_element = 0            
        elif super_square_element_or_column in [1,4,7]: column_or_super_square_element = 1            
        elif super_square_element_or_column in [2,5,8]: column_or_super_square_element = 2            
            
    if super_square_or_row in [1,4,7]:
        if super_square_element_or_column in [0,3,6]: column_or_super_square_element = 3            
        elif super_square_element_or_column in [1,4,7]: column_or_super_square_element = 4            
        elif super_square_element_or_column in [2,5,8]: column_or_super_square_element = 5            
            
    if super_square_or_row in [2,5,8]:
        if super_square_element_or_column in [0,3,6]: column_or_super_square_element = 6            
        elif super_square_element_or_column in [1,4,7]: column_or_super_square_element = 7            
        elif super_square_element_or_column in [2,5,8]: column_or_super_square_element = 8
            
    return column_or_super_square_element

# this function also takes advantage of shared relationship:
# 1. determines the super square given a row and column
# 2. determines the row given a super square and super square element 
def identify_row_or_super_square(row_or_super_square, column_super_square_element):
    if row_or_super_square in [0,1,2]:
        if column_super_square_element in [0,1,2]: super_square_or_row = 0            
        if column_super_square_element in [3,4,5]: super_square_or_row = 1            
        if column_super_square_element in [6,7,8]: super_square_or_row = 2            
            
    if row_or_super_square in [3,4,5]:
        if column_super_square_element in [0,1,2]: super_square_or_row = 3            
        if column_super_square_element in [3,4,5]: super_square_or_row = 4            
        if column_super_square_element in [6,7,8]: super_square_or_row = 5            

    if row_or_super_square in [6,7,8]:
        if column_super_square_element in [0,1,2]: super_square_or_row = 6            
        if column_super_square_element in [3,4,5]: super_square_or_row = 7            
        if column_super_square_element in [6,7,8]:super_square_or_row = 8  
                      
    return super_square_or_row

# the following is a blank grid before the actual sudoku grids, as copied from sudoku.com
blank = {"row0": [None,None,None,None,None,None,None,None,None], 
        "row1" : [None,None,None,None,None,None,None,None,None],
        "row2" : [None,None,None,None,None,None,None,None,None],
        "row3" : [None,None,None,None,None,None,None,None,None],
        "row4" : [None,None,None,None,None,None,None,None,None],
        "row5" : [None,None,None,None,None,None,None,None,None],
        "row6" : [None,None,None,None,None,None,None,None,None],
        "row7" : [None,None,None,None,None,None,None,None,None],
        "row8" : [None,None,None,None,None,None,None,None,None]}

easy = {"row0" : [None,None,None,1,None,2,None,3,7], 
        "row1" : [None,None,None,4,None,9,5,8,6],
        "row2" : [None,None,6,None,None,8,9,None,None],
        "row3" : [None,7,None,None,None,None,8,6,5],
        "row4" : [None,None,1,6,None,None,None,None,3],
        "row5" : [6,3,None,2,None,None,None,None,1],
        "row6" : [8,1,7,3,4,5,6,2,9],
        "row7" : [3,9,None,None,2,None,None,5,None],
        "row8" : [None,6,5,None,None,None,None,None,None]}

medium0 ={"row0":[None,6,None,None,None,7,None,1,None], 
        "row1" : [None,None,None,None,1,None,None,8,None],
        "row2" : [None,9,None,None,None,3,2,5,None],
        "row3" : [7,3,None,5,9,None,None,6,2],
        "row4" : [2,None,None,None,6,None,None,None,None],
        "row5" : [None,8,None,None,3,None,4,None,9],
        "row6" : [9,2,None,None,None,1,3,None,None],
        "row7" : [None,None,None,None,4,None,7,None,None],
        "row8" : [None,7,None,None,None,None,8,None,1]}

medium1 ={"row0":[2,None,None,None,6,4,None,None,None], 
        "row1" : [None,None,None,None,9,None,6,None,1],
        "row2" : [6,8,5,1,3,None,None,None,None],
        "row3" : [5,None,None,None,None,7,1,None,9],
        "row4" : [None,None,None,None,None,5,None,None,6],
        "row5" : [None,None,7,6,None,None,5,4,2],
        "row6" : [None,None,None,8,None,None,None,None,None],
        "row7" : [None,None,1,3,4,None,None,None,7],
        "row8" : [4,None,3,None,None,6,None,None,None]}

medium2 ={"row0":[1,7,None,None,None,None,None,None,6], 
        "row1" : [3,6,None,None,None,None,1,None,None],
        "row2" : [2,None,None,1,None,9,None,7,None],
        "row3" : [None,3,6,None,None,None,8,1,None],
        "row4" : [4,5,None,3,None,7,None,None,None],
        "row5" : [None,8,None,5,None,None,7,None,None],
        "row6" : [None,None,4,None,None,1,None,None,None],
        "row7" : [None,1,None,2,None,None,4,None,None],
        "row8" : [None,2,None,9,None,8,None,5,None]}

hard0 = {"row0": [None,None,None,None,5,None,None,None,4], 
        "row1" : [None,7,None,None,None,6,None,None,None],
        "row2" : [None,1,None,None,None,None,3,None,None],
        "row3" : [None,None,8,4,1,None,None,6,None],
        "row4" : [None,None,9,None,None,None,5,None,None],
        "row5" : [None,None,1,None,2,8,None,7,None],
        "row6" : [None,None,None,None,7,None,1,None,5],
        "row7" : [None,2,None,8,None,1,None,None,None],
        "row8" : [None,None,None,None,4,9,None,2,None]}

hard1 = {"row0": [None,None,3,7,None,None,None,None,8], 
        "row1" : [None,2,None,9,None,None,None,1,None],
        "row2" : [None,None,None,None,None,None,None,None,None],
        "row3" : [None,3,None,None,None,7,5,None,None],
        "row4" : [None,4,None,1,None,None,None,7,None],
        "row5" : [None,None,None,None,5,6,None,None,None],
        "row6" : [None,None,8,6,9,None,1,None,None],
        "row7" : [None,None,6,None,None,None,None,5,None],
        "row8" : [None,None,9,3,None,None,None,2,None]}

master = {"row0":[None,None,5,None,None,8,3,9,None], 
        "row1" : [None,3,None,None,None,None,None,None,None],
        "row2" : [None,None,None,7,None,None,None,8,None],
        "row3" : [None,None,4,5,None,None,6,None,2],
        "row4" : [6,1,None,None,None,None,None,None,None],
        "row5" : [2,None,None,None,4,9,None,None,None],
        "row6" : [None,None,None,None,None,2,4,None,5],
        "row7" : [None,None,9,None,8,None,None,None,None],
        "row8" : [5,6,None,None,None,None,None,None,None]}
    
evil = {"row0" : [None,7,None,8,None,None,2,None,None], 
        "row1" : [None,None,None,5,None,None,None,None,1],
        "row2" : [None,None,4,None,7,1,3,None,None],
        "row3" : [None,5,None,None,2,9,None,None,3],
        "row4" : [None,None,None,1,None,None,None,None,None],
        "row5" : [6,None,None,None,None,None,4,None,None],
        "row6" : [None,None,5,None,None,None,None,2,None],
        "row7" : [None,None,None,None,8,None,None,None,None],
        "row8" : [None,2,None,None,3,7,None,None,9]}

# setting the sudoku grid to be solved
rows = medium1

# setting if solving progress will be displayed or not
verbose = False

# this transposes the above rows into a dictionary of the columns
columns = {f"column{i}": [rows[j][i] for j in rows] for i in range(0,9)}
   
# this generates a blank grid for the super squares
super_squares = {f"super_square{i}": [] for i in range(9)}
# this function finds the correct rows for each super square. The first three super squares (starting from 
# top-left finishing bottom-right) correspond to rows 0-2, the next three super squares correspond to rows 3-5, etc. 
def find_super_row(element): # "super row" being the three sets of rows each made of 9x3 squares
    if element < 3:
        a = 0
        b = 3
        return [a,b]
    elif element < 6:
        a = 3
        b = 6
        return [a,b]
    elif element < 9:
        a = 6
        b = 9
        return [a,b]    
# the script will calculate the super squares using the rows data. After the correct rows are found,
# this function finds the correct elements in that row. E.g., the 2nd element (from top-left to 
# bottom-right) needs the middle 3 elements (of the top 3 rows)
def locate_row_section(element):
    location = []
    if element in [0,3,6]: location = [0,1,2]        
    if element in [1,4,7]: location = [3,4,5]        
    if element in [2,5,8]: location = [6,7,8]
    return location    
# this finally creates the super squares
for element in range(0,9):
    a = find_super_row(element)[0]
    b = find_super_row(element)[1]
    for row_no in range(a,b):
        for i in range(3):
            super_squares[f"super_square{element}"].append(rows[f"row{row_no}"][locate_row_section(element)[i]])
 
         
# with the sudoku grids generated and their relationships captured, now it solves them starting arbitrarily with super squares
# this starts by checking for solo candidates before checking for hidden singles, repeating until neither method produces any answers
def super_squares_routine(verbose, super_squares, rows, columns): 
    if verbose: print("\n"), print("Starting super squares routine")
        
    main_iter_count = 0
    while True:
        at_least_one_answer_found = False
        main_iter_count += 1
        if verbose: print("\n"), print(f"Main iteration {main_iter_count} for super squares routine"), print("\n")  

        # loop which detects solo candidates until a search of all super squares and their elements produce no answers
        for super_square_iterator in range(9): # iterator over all nine super squares from top-left to bottom-right
            if verbose: print("\n"), print(f"Searching for solo candidates - super square {super_square_iterator}"), print("\n")
            current_super_square = super_squares[f"super_square{super_square_iterator}"]
            
            # now in a super square, this loops through the nine elements in said square, which also goes from top-left to bottom-right
            for element_iterator in range(9):
                
                if verbose: print("\n"), print(f"Super square {super_square_iterator}, element: {element_iterator}"), print("\n") 
                
                # if there's already an answer then this skips the loop to the next element
                current_answer = super_squares[f"super_square{super_square_iterator}"][element_iterator]
                if current_answer != None:
                    if verbose: print(f"Answer already present: {current_answer}")
                    continue           

                # an element's potential answers are made as a list from 1-9, excepting already known answers
                corresponding_row = identify_row_or_super_square(super_square_iterator, element_iterator)
                corresponding_column = identify_column_or_super_square_element(super_square_iterator, element_iterator)
                potential_answers = [i for i in range(1,10) 
                                     if i not in current_super_square 
                                     and i not in rows[f"row{corresponding_row}"] 
                                     and i not in columns[f"column{corresponding_column}"]]
                if verbose: print(f"Potential answers after accounting for already known answers: {potential_answers}")

                # if there is only one potential answer remaining, then this must be the solo candidate and thus is stored as the answer
                if len(potential_answers) == 1:
                    if verbose: print(f"Solo candidate answer found: {potential_answers[0]}, in super square {super_square_iterator}, element {element_iterator}")
                    super_squares[f"super_square{super_square_iterator}"][element_iterator] = potential_answers[0]
                    rows[f"row{corresponding_row}"][corresponding_column] = potential_answers[0]
                    columns[f"column{corresponding_column}"][corresponding_row] = potential_answers[0]
                    at_least_one_answer_found = True
        

        # if the solo candidates routine completes the whole grid without finding a single answer
        # then the following routine searches for hidden singles
        if not at_least_one_answer_found:         
            for super_square_iterator_H in range(9): # again iterating over each super square                
                if verbose: print("\n"), print(f"Searching for hidden singles - super square {super_square_iterator_H}"), print("\n")
                
                candidates = [] # candidates is a list of lists where each sub list represents an element
                
                for element_iterator_H in range(9): # iterating over each element within the super square
                    if verbose: print("\n"), print(f"Super square {super_square_iterator_H}, element {element_iterator_H}"), print("\n")
                    
                    # sees if there is already an answer and sets it to that, then skipping the loop
                    if super_squares[f"super_square{super_square_iterator_H}"][element_iterator_H] != None:
                        candidates.append([super_squares[f"super_square{super_square_iterator_H}"][element_iterator_H]])
                        continue                        
                    
                    # as in the solo candidates routine, potential answers start as a list 1-9, excluding already present answers
                    corresponding_column_of_super_square_H = identify_column_or_super_square_element(super_square_iterator_H, element_iterator_H)
                    corresponding_row_of_super_square_H = identify_row_or_super_square(super_square_iterator_H, element_iterator_H)
                    potential_answers_H = [i for i in range(1,10) 
                                           if i not in super_squares[f"super_square{super_square_iterator_H}"] 
                                           and i not in columns[f"column{corresponding_column_of_super_square_H}"] 
                                           and i not in rows[f"row{corresponding_row_of_super_square_H}"]]
                    candidates.append(potential_answers_H)                                        
                
                # the following flattens candidates into a single list
                flat_candidates = [item for sub_list in candidates for item in sub_list]
                unique_candidates = set(flat_candidates)
                if verbose: print(f"Potential answers after accounting for already known answers: {candidates}")
                    
                # this detects if a potential answer appears only once in a super square's candidates, meaning it is a hidden single
                for unique_candidate in unique_candidates:
                    if flat_candidates.count(unique_candidate) == 1: # i.e., if a certain number is a hidden single
                        for super_square_element_H, candidate in enumerate(candidates):                            
                            if unique_candidate in candidate: # i.e., if the given element is the hidden single                                
                                if super_squares[f"super_square{super_square_iterator_H}"][super_square_element_H] == None: # skips already found answers
                                    
                                    at_least_one_answer_found = True
                                    
                                    if verbose: print(f"Hidden answer found: {unique_candidate}, in super square {super_square_iterator_H}, element {super_square_element_H}")
                                    
                                    super_squares[f"super_square{super_square_iterator_H}"][super_square_element_H] = unique_candidate                                
                                    
                                    column_H = identify_column_or_super_square_element(super_square_iterator_H, super_square_element_H)
                                    row_H = identify_row_or_super_square(super_square_iterator_H, super_square_element_H)
                                    rows[f"row{row_H}"][column_H] = unique_candidate
                                    columns[f"column{column_H}"][row_H] = unique_candidate                                                                                                           
                                    
            # if neither the solo candidates nor the hidden singles routine finds a single answer,
            # then the sudoku grid is saved so it can be compared to before and after the function ran
            # to see if it made any progress
            if not at_least_one_answer_found:
                if verbose: print("\n"), print("Full loop of super squares completed without finding an answer")
                grid_state_at_end_of_routine = copy.deepcopy(rows)
                return grid_state_at_end_of_routine
            

# now solving by rows, and as before this starts by checking for solo candidates singles before
# checking for hidden singles, repeating until neither method produces an answer
def row_routine(verbose, rows, columns, super_squares):
    if verbose: print("\n"), print("Starting rows routine")
    
    main_iter_count = 0
    while True:
        at_least_one_answer_found = False
        main_iter_count += 1
        if verbose: print("\n"), print(f"Main iteration {main_iter_count} for row routine"), print("\n")  
        
        # loop which detects solo candidates until a search of all rows and their elements produce no answers
        for row_iterator in range(9): # iterator over all nine rows from top to bottom
            if verbose: print("\n"), print(f"Searching for solo candidates - row {row_iterator}"), print("\n")
            current_row = rows[f"row{row_iterator}"]

            # with a row selected, this loops through the nine elements in said row, also from top to bottom
            for element_iterator in range(9): 
                
                # note that for rows, the element number is the same as the column number
                if verbose: print("\n"), print(f"Row: {row_iterator}, element/column: {element_iterator}"), print("\n")
                
                # if there's already an answer then this skips the loop to the next element
                current_answer = rows[f"row{row_iterator}"][element_iterator]
                if current_answer != None:
                    if verbose: print(f"Answer already present: {current_answer}")
                    continue            

                # an element's potential answers are made as a list from 1-9, excepting already known answers
                corresponding_super_square = identify_row_or_super_square(row_iterator, element_iterator)
                potential_answers = [i for i in range(1,10) 
                                     if i not in current_row 
                                     and i not in columns[f"column{element_iterator}"] 
                                     and i not in super_squares[f"super_square{corresponding_super_square}"]]
                if verbose: print(f"Potential answers after accounting for already known answers: {potential_answers}")
                
                # if there is only one potential answer remaining, then this must be the solo candidate and thus is stored as the answer 
                if len(potential_answers) == 1:
                    
                    if verbose: print(f"Solo candidate answer found: {potential_answers[0]}, in row {row_iterator}, element/column {element_iterator}")
                        
                    columns[f"column{element_iterator}"][row_iterator] = potential_answers[0] 
                    rows[f"row{row_iterator}"][element_iterator] = potential_answers[0]
                      
                    super_square_element = identify_row_or_super_square_element(element_iterator, row_iterator) 
                    super_squares[f"super_square{corresponding_super_square}"][super_square_element] = potential_answers[0]
                    
                    at_least_one_answer_found = True                     

        # if the solo candidates routine completes the whole grid without finding a single answer
        # then the following routine searches for hidden singles
        if not at_least_one_answer_found:            
            for row_iterator_H in range(9): # again iterating over each row 
                if verbose: print("\n"), print(f"Searching for hidden singles - row {row_iterator_H}"), print("\n")
                
                candidates = [] # candidates is a list of lists where each sub list represents an element
                
                for element_iterator_H in range(9): # iterating over each element within the row
                    if verbose: print("\n"), print(f"Row {row_iterator_H}, element {element_iterator_H}"), print("\n")
                    
                    # sees if there is already an answer and sets it to that, then skipping the loop
                    if rows[f"row{row_iterator_H}"][element_iterator_H] != None:
                        candidates.append([rows[f"row{row_iterator_H}"][element_iterator_H]])
                        continue                        
                    
                    # as in the solo candidates routine, potential answers start as a list 1-9, excluding already present answers
                    super_square_H = identify_row_or_super_square(row_iterator_H, element_iterator_H) 
                    potential_answers_H = [i for i in range(1,10) 
                                           if i not in rows[f"row{row_iterator_H}"] 
                                           and i not in columns[f"column{element_iterator_H}"] 
                                           and i not in super_squares[f"super_square{super_square_H}"]]
                    candidates.append(potential_answers_H)                    

                # the following flattens candidates into a single list
                flat_candidates = [item for sub_list in candidates for item in sub_list]
                unique_candidates = set(flat_candidates)
                if verbose: print(f"Potential answers after accounting for already known answers: {candidates}")
                
                # this detects if a potential answer appears only once in a row's candidates, meaning it is a hidden single
                for unique_candidate in unique_candidates:
                    if flat_candidates.count(unique_candidate) == 1: # i.e., if a certain number is a hidden single
                        for column_H, candidate in enumerate(candidates):                            
                            if unique_candidate in candidate: # i.e., if the given element is the hidden single       
                                if rows[f"row{row_iterator_H}"][column_H] == None: # skips already found answers
                                    
                                    at_least_one_answer_found = True
                                    
                                    if verbose: print(f"Hidden answer found: {unique_candidate}, in row {row_iterator_H}, element/column {column_H}")
                                    
                                    rows[f"row{row_iterator_H}"][column_H] = unique_candidate
                                    
                                    columns[f"column{column_H}"][row_iterator_H] = unique_candidate
                                    
                                    super_square_element_H = identify_row_or_super_square_element(column_H, row_iterator_H)
                                    super_square_H = identify_row_or_super_square(row_iterator_H, column_H)
                                    super_squares[f"super_square{super_square_H}"][super_square_element_H] = unique_candidate

            # if neither the solo candidates nor the hidden singles routine finds a single answer,
            # then the sudoku grid is saved so it can be compared to before and after the function ran
            # to see if it made any progress
            if not at_least_one_answer_found:
                if verbose: print("\n"), print("Full loop of rows completed without finding an answer")
                grid_state_at_end_of_routine = copy.deepcopy(rows)
                return grid_state_at_end_of_routine


# finally solving by columns, and as before this starts by checking for solo candidates singles before
# checking for hidden singles, repeating until neither method produces an answer
def column_routine(verbose, columns, rows, super_squares):
    if verbose: print("\n"), print("Starting columns routine")
                      
    main_iter_count = 0
    while True:
        at_least_one_answer_found = False
        main_iter_count += 1
        if verbose: print("\n"), print(f"Main iteration {main_iter_count} for column routine"), print("\n")
        
        # loop which detects solo candidates until a search of all columns and their elements produce no answers
        for column_iterator in range(9): # iterator over all nine columns from left to right
            if verbose: print("\n"), print(f"Searching for solo candidates - column {column_iterator}")
            current_column = columns[f"column{column_iterator}"]

            # now in a column, this loops through the nine elements in column, which also goes from left to right
            for element_iterator in range(9):
                # note that for columns, the element number is the same as the row number
                if verbose: print("\n"), print(f"Column {column_iterator}, element/row {element_iterator}")                

                # if there's already an answer then this skips the loop to the next element
                current_answer = columns[f"column{column_iterator}"][element_iterator]
                if current_answer != None:
                    if verbose: print(f"Answer already present: {current_answer}")
                    continue            

                # all numbers 1-9 are added as a potential answer, except already known answers
                corresponding_super_square = column_routine__identify_super_square(column_iterator, element_iterator)
                potential_answers = [i for i in range(1,10) 
                                     if i not in current_column 
                                     and i not in rows[f"row{element_iterator}"] 
                                     and i not in super_squares[f"super_square{corresponding_super_square}"]]
                if verbose: print(f"Potential answers after accounting for already known answers: {potential_answers}")
                
                # if there is only one potential answer remaining, then this must be the solo candidate and thus is stored as the answer
                if len(potential_answers) == 1:
                    columns[f"column{column_iterator}"][element_iterator] = potential_answers[0]
                    
                    rows[f"row{element_iterator}"][column_iterator] = potential_answers[0]
                    
                    super_square_element = identify_column_or_super_square_element(element_iterator, column_iterator)
                    super_squares[f"super_square{corresponding_super_square}"][super_square_element] = potential_answers[0]
                    
                    at_least_one_answer_found = True
                    
                    if verbose: print(f"Solo candidate answer found: {potential_answers[0]}, in column {column_iterator}, element/row {element_iterator}")               
        
        # if the solo candidates routine completes the whole grid without finding a single answer
        # then the following routine searches for hidden singles
        if not at_least_one_answer_found:         
            for column_iterator_H in range(9): # again iterating over each column                
                if verbose: print("\n"), print(f"Searching for hidden singles - column {column_iterator_H}"), print("\n")
                
                candidates = [] # candidates is a list of lists where each sub list represents an element                
                
                for element_iterator_H in range(9): # iterating over each element within the column
                    if verbose: print("\n"), print(f"Row {column_iterator_H}, element {element_iterator_H}"), print("\n")
                    
                    # sees if there is already an answer and sets it to that, then skipping the loop
                    if columns[f"column{column_iterator_H}"][element_iterator_H] != None:
                        candidates.append([columns[f"column{column_iterator_H}"][element_iterator_H]])
                        continue                        
                    
                    # as in the solo candidates routine, potential answers start as a list 1-9, excluding already present answers
                    super_square_H = column_routine__identify_super_square(column_iterator_H, element_iterator_H) 
                    potential_answers_H = [i for i in range(1,10) 
                                           if i not in columns[f"column{column_iterator_H}"] 
                                           and i not in rows[f"row{element_iterator_H}"] 
                                           and i not in super_squares[f"super_square{super_square_H}"]]
                    candidates.append(potential_answers_H)                     

                # the following flattens candidates into a single list
                flat_candidates = [item for sub_list in candidates for item in sub_list]
                unique_candidates = set(flat_candidates)
                if verbose: print(f"Potential answers after accounting for already known answers {candidates}")
                
                # this detects if a potential answer appears only once in a super column's candidates, meaning it is a hidden single
                for unique_candidate in unique_candidates:
                    if flat_candidates.count(unique_candidate) == 1: # i.e., if a certain number is a hidden single
                        for row_H, candidate in enumerate(candidates):
                            if unique_candidate in candidate: # i.e., if the given element is the hidden single  
                                if columns[f"column{column_iterator_H}"][row_H] == None: # skips already found answers
                                    
                                    at_least_one_answer_found = True
                                    
                                    if verbose: print(f"Hidden answer found: {unique_candidate}, in column {column_iterator_H}, element/row {element_iterator_H}")
                                    
                                    rows[f"row{row_H}"][column_iterator_H] = unique_candidate
                                    
                                    columns[f"column{column_iterator_H}"][row_H] = unique_candidate
                                    
                                    super_square_element_H = identify_column_or_super_square_element(row_H, column_iterator_H)
                                    super_square_H = column_routine__identify_super_square(column_iterator_H, row_H)
                                    super_squares[f"super_square{super_square_H}"][super_square_element_H] = unique_candidate

            # if neither the solo candidates nor the hidden singles routine finds a single answer,
            # then the sudoku grid is saved so it can be compared to before and after the function ran
            # to see if it made any progress
            if not at_least_one_answer_found:
                if verbose: print("\n"), print("Full loop of columns completed without finding an answer")
                grid_state_at_end_of_routine = copy.deepcopy(rows)
                return grid_state_at_end_of_routine

            
# this functions verifies if a sudoku grid has been completed by checking that all elements have numbers
def check_if_finished():
    sudoku_grid_flattened = [value for key, value in rows.items()]
    sudoku_grid_flattened = [i for entry in sudoku_grid_flattened for i in entry]
    if None in sudoku_grid_flattened:
        finished = False
    else:
        finished = True    
    return finished    


# with everything set up, the below runs the solving routines as appropriate
routines = [super_squares_routine, row_routine, column_routine] # this list used by the loop below features the three solving functions' names
# splitting of function names and variables in these two lists instead of just having a list of the
# functions is necessary to avoid calling them before the loop below starts
variables = [(verbose, super_squares, rows, columns),
            (verbose, rows, columns, super_squares),
            (verbose, columns, rows, super_squares)] 

# this runs through the above routines to complete the sudoku grid, arbitrarily going from super squares,
# rows, and then columns. It does this by comparing the sudoku grid before and after running a routine. If
# all three routines produce not a single answer then it exits
while True:
    progress_made = False
    for count, routine in enumerate(routines):
        if check_if_finished(): break
        grid_prestate = copy.deepcopy(rows) # saving grid prior to running routine      
        grid_state_at_end_of_routine = routine(*variables[count]) # runs routine and saves the grid it produces
        # checks that the grid before and after are the same, if not then at least one answer was found, hence progress_made = True
        if grid_prestate != grid_state_at_end_of_routine: 
            progress_made = True # this is not reset until a full loop of all three routines
        # if the final column routine has run and none of the three routines found an answer, then it quits
        if routine == column_routine and not progress_made:
            if verbose: print("\n")
            print("All three routines unable to find answers")
            break      

    if check_if_finished(): 
        if verbose: print("\n")
        print("sudoku complete!")
        break
        
    # if it does the final column routine but it did find at least one answer, then it restarts to see if there are more
    if routine == column_routine and progress_made: continue
        
    break # used if all three routines cannot find an answer

sudoku complete!


In [5]:
# the following is a check that its solutions are correct; the sum of a completed sudoku grid is 405
routines = [rows,columns,super_squares]
routines_names = ["rows","columns","super_squares"]

for name, routine in enumerate(routines):
    sudoku_grid_flattened = [value for key, value in routine.items()]
    for i in range(len(sudoku_grid_flattened)):
        for j in range(9):
            if sudoku_grid_flattened[i].count(j) > 1:
                print(f"{routines_names[name]} error: more than one instance of {j} found in {sudoku_grid_flattened[i]}")
    sudoku_grid_flattened = [i for entry in sudoku_grid_flattened for i in entry]
    removed_nones = [i for i in sudoku_grid_flattened if i != None]
    print(f"{routines_names[name]}: {sum(removed_nones)}")

rows: 405
columns: 405
super_squares: 405


#### Sudoku grid pre-script:

In [2]:
rows

{'row0': [2, None, None, None, 6, 4, None, None, None],
 'row1': [None, None, None, None, 9, None, 6, None, 1],
 'row2': [6, 8, 5, 1, 3, None, None, None, None],
 'row3': [5, None, None, None, None, 7, 1, None, 9],
 'row4': [None, None, None, None, None, 5, None, None, 6],
 'row5': [None, None, 7, 6, None, None, 5, 4, 2],
 'row6': [None, None, None, 8, None, None, None, None, None],
 'row7': [None, None, 1, 3, 4, None, None, None, 7],
 'row8': [4, None, 3, None, None, 6, None, None, None]}

#### Sudoku grid post-script:

In [4]:
rows

{'row0': [2, 1, 9, 7, 6, 4, 3, 8, 5],
 'row1': [7, 3, 4, 5, 9, 8, 6, 2, 1],
 'row2': [6, 8, 5, 1, 3, 2, 7, 9, 4],
 'row3': [5, 6, 8, 4, 2, 7, 1, 3, 9],
 'row4': [3, 4, 2, 9, 1, 5, 8, 7, 6],
 'row5': [1, 9, 7, 6, 8, 3, 5, 4, 2],
 'row6': [9, 2, 6, 8, 7, 1, 4, 5, 3],
 'row7': [8, 5, 1, 3, 4, 9, 2, 6, 7],
 'row8': [4, 7, 3, 2, 5, 6, 9, 1, 8]}